### 加载 Cookies 并访问页面的函数

In [12]:
import pickle
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

# 初始化 Edge 浏览器
def init_edge_browser():
    driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
    return driver

# 从文件中加载 Cookies 并添加到 Selenium
def load_cookies(driver, cookies_file_path='cookies.pkl'):
    with open(cookies_file_path, 'rb') as f:
        cookies = pickle.load(f)

    # 打开一个初始页面，以便 Selenium 可以加载 Cookies
    driver.get("https://pintia.cn")
    time.sleep(3)  # 等待页面加载

    # 将每个 cookie 加入到浏览器
    for cookie in cookies:
        driver.add_cookie(cookie)

    print("Cookies 已加载到浏览器中。")

# 使用 Selenium 打开页面并加载树状知识点
def load_page_and_expand_knowledge_tree(driver, url):
    driver.get(url)
    time.sleep(5)  # 等待页面加载完成

    # 找到所有知识点的展开按钮（假设有展开按钮）
    expand_buttons = driver.find_elements_by_css_selector('span.tree-toggle')
    for button in expand_buttons:
        button.click()  # 展开所有知识点
        time.sleep(1)   # 每次点击后稍作等待，确保页面完成加载
    print("所有知识点已展开")


### 爬取知识点和题目

In [13]:
from bs4 import BeautifulSoup

# 获取每个知识点及其题目
def get_knowledge_points_and_questions(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 获取左侧知识点树中的每个最小分支
    knowledge_points = soup.select('.tree-node')  # 选择知识点节点
    knowledge_data = {}

    for kp in knowledge_points:
        knowledge_name = kp.get_text().strip()  # 获取知识点名称

        # 点击该知识点，获取对应的题目
        kp_element = driver.find_element_by_link_text(knowledge_name)
        kp_element.click()
        time.sleep(3)  # 等待页面加载题目

        # 获取题目信息
        questions_soup = BeautifulSoup(driver.page_source, 'html.parser')
        questions = questions_soup.select('.question-row')  # 选择题目行
        question_list = []

        for question in questions:
            title = question.select_one('.question-title').get_text().strip()
            difficulty = question.select_one('.difficulty-level').get_text().strip()
            source = question.select_one('.source').get_text().strip()
            citations = question.select_one('.citation-count').get_text().strip()

            # 保存题目信息
            question_list.append({
                'title': title,
                'difficulty': difficulty,
                'source': source,
                'citations': citations
            })

        knowledge_data[knowledge_name] = question_list

    return knowledge_data
from bs4 import BeautifulSoup

# 获取每个知识点及其题目
def get_knowledge_points_and_questions(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 获取左侧知识点树中的每个最小分支
    knowledge_points = soup.select('.tree-node')  # 选择知识点节点
    knowledge_data = {}

    for kp in knowledge_points:
        knowledge_name = kp.get_text().strip()  # 获取知识点名称

        # 点击该知识点，获取对应的题目
        kp_element = driver.find_element_by_link_text(knowledge_name)
        kp_element.click()
        time.sleep(3)  # 等待页面加载题目

        # 获取题目信息
        questions_soup = BeautifulSoup(driver.page_source, 'html.parser')
        questions = questions_soup.select('.question-row')  # 选择题目行
        question_list = []

        for question in questions:
            title = question.select_one('.question-title').get_text().strip()
            difficulty = question.select_one('.difficulty-level').get_text().strip()
            source = question.select_one('.source').get_text().strip()
            citations = question.select_one('.citation-count').get_text().strip()

            # 保存题目信息
            question_list.append({
                'title': title,
                'difficulty': difficulty,
                'source': source,
                'citations': citations
            })

        knowledge_data[knowledge_name] = question_list

    return knowledge_data


In [14]:
def handle_pagination(driver):
    # 获取分页数
    while True:
        try:
            next_button = driver.find_element(By.LINK_TEXT, '下一页')
            next_button.click()
            time.sleep(3)  # 等待加载下一页题目
            # 爬取下一页题目
        except:
            # 没有下一页了
            break


### 将爬取的数据保存为 JSON 或 CSV

In [15]:
import json

def save_data_to_json(knowledge_data, file_name='data.json'):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(knowledge_data, f, ensure_ascii=False, indent=4)
    print(f"数据已保存到 {file_name}")


### 执行流程

In [18]:
if __name__ == "__main__":
    # 初始化 Edge 浏览器
    driver = init_edge_browser()

    # 加载保存的 Cookies
    load_cookies(driver)

    # 打开题库页面并展开知识点
    url = "https://pintia.cn/problems?knowledgePointId"  # 替换为实际的题库页面
    load_page_and_expand_knowledge_tree(driver, url)

    # 获取知识点和题目信息
    knowledge_data = get_knowledge_points_and_questions(driver)

    # 保存数据到 JSON 文件
    save_data_to_json(knowledge_data)

    # 关闭浏览器
    driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=130.0.2849.46)
Stacktrace:
	GetHandleVerifier [0x00007FF76E4FDC75+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76E7A83C4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76E6E76A6+1460486]
	(No symbol) [0x00007FF76E2984C7]
	(No symbol) [0x00007FF76E31CBED]
	(No symbol) [0x00007FF76E330D7A]
	(No symbol) [0x00007FF76E318163]
	(No symbol) [0x00007FF76E2EFB54]
	(No symbol) [0x00007FF76E2EF000]
	(No symbol) [0x00007FF76E2EF741]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76E67CDB4+1024020]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF76E419A04+57108]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76E67BDB3+1019923]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF76E67BA19+1019001]
	Microsoft::Applications::Event

In [25]:
import requests
import pickle
import json

# 从文件中加载 Cookies
def load_cookies_from_file(cookies_file_path='cookies.pkl'):
    with open(cookies_file_path, 'rb') as f:
        cookies = pickle.load(f)
    return cookies

# 使用保存的 Cookies 发送 API 请求
def request_with_cookies(api_url, cookies_file_path='cookies.pkl'):
    # 加载 Cookies
    cookies = load_cookies_from_file(cookies_file_path)

    # 创建 session，使用保存的 cookies
    session = requests.Session()

    # 将 cookies 添加到 session
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    # 设置请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json;charset=UTF-8',
        'Referer': 'https://pintia.cn/problems?knowledgePointId=63',
        'Content-Type': 'application/json;charset=UTF-8'
    }

    # 发送请求
    response = session.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"请求失败，状态码: {response.status_code}")
        return None

# 递归抓取知识点及其子知识点
def fetch_knowledge_points(parent_id, cookies_file_path='cookies.pkl', knowledge_points=[]):
    api_url = f"https://pintia.cn/api/knowledge-points?parent_id={parent_id}&limit=200"
    data = request_with_cookies(api_url, cookies_file_path)

    if data:
        # 遍历每个 knowledgePointPath 中的知识点
        for kp_path in data.get('knowledgePointPaths', []):
            for knowledge_point in kp_path.get('knowledgePoints', []):
                # 确保不再重复爬取同一知识点
                if knowledge_point['id'] not in [kp['id'] for kp in knowledge_points]:
                    print(f"知识点: {knowledge_point['name']} (ID: {knowledge_point['id']})")
                    knowledge_points.append(knowledge_point)

                    # 如果不是叶子节点，继续递归获取子节点
                    if not knowledge_point['isLeaf']:
                        fetch_knowledge_points(knowledge_point['id'], cookies_file_path, knowledge_points)

    return knowledge_points

# 保存数据到 JSON 文件
def save_data_to_json(data, file_name='knowledge_points.json'):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"数据已保存到 {file_name}")

# 使用示例
if __name__ == "__main__":
    root_parent_id = 63  # C程序设计的根节点ID
    all_knowledge_points = fetch_knowledge_points(root_parent_id)

    if all_knowledge_points:
        save_data_to_json(all_knowledge_points)


知识点: C程序设计 (ID: 63)
知识点: 基本数据类型与表达式 (ID: 64)
知识点: 表达式 (ID: 65)
知识点: 输入输出格式化控制 (ID: 66)
知识点: 分支控制 (ID: 67)
知识点: if-else (ID: 68)
知识点: switch (ID: 69)
知识点: 循环控制 (ID: 70)
知识点: for (ID: 71)
知识点: while和do-while (ID: 72)
知识点: 嵌套循环 (ID: 73)
知识点: 函数与程序结构 (ID: 74)
知识点: 函数定义与调用 (ID: 75)
知识点: 递归函数 (ID: 76)
知识点: 变量作用域与存储类型 (ID: 77)
知识点: 数组 (ID: 78)
知识点: 一维数组 (ID: 79)
知识点: 字符串 (ID: 80)
知识点: 二维数组 (ID: 81)
知识点: 指针与结构 (ID: 82)
知识点: 指针 (ID: 83)
知识点: 结构 (ID: 84)
知识点: 链表 (ID: 85)
知识点: 文件 (ID: 86)
知识点: 文本文件处理 (ID: 87)
知识点: 二进制文件处理 (ID: 88)
知识点: 其他 (ID: 89)
知识点: 编译预处理 (ID: 90)
知识点: 命令行参数 (ID: 91)
知识点: 其他 (ID: 315)
知识点: 枚举 (ID: 1058957573250904064)
知识点: 共同体 (ID: 1058957606629179392)
数据已保存到 knowledge_points.json


In [28]:
import requests
import pickle
import json

# 从文件中加载 Cookies
def load_cookies_from_file(cookies_file_path='cookies.pkl'):
    with open(cookies_file_path, 'rb') as f:
        cookies = pickle.load(f)
    return cookies

# 使用保存的 Cookies 发送 API 请求
def request_with_cookies(api_url, cookies_file_path='cookies.pkl'):
    # 加载 Cookies
    cookies = load_cookies_from_file(cookies_file_path)

    # 创建 session，使用保存的 cookies
    session = requests.Session()

    # 将 cookies 添加到 session
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    # 设置请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json;charset=UTF-8',
        'Referer': 'https://pintia.cn/problems?knowledgePointId=63',
        'Content-Type': 'application/json;charset=UTF-8'
    }

    # 发送请求
    response = session.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"请求失败，状态码: {response.status_code}")
        return None

# 递归抓取知识点及其子知识点，并构建层级结构
def fetch_knowledge_points(parent_id, cookies_file_path='cookies.pkl', processed_ids=None):
    if processed_ids is None:
        processed_ids = set()  # 用于记录已处理的知识点ID，防止重复抓取

    api_url = f"https://pintia.cn/api/knowledge-points?parent_id={parent_id}&limit=200"
    data = request_with_cookies(api_url, cookies_file_path)
    knowledge_tree = []

    if data:
        # 遍历每个 knowledgePointPath 中的知识点
        for kp_path in data.get('knowledgePointPaths', []):
            for knowledge_point in kp_path.get('knowledgePoints', []):
                # 如果该知识点已经被处理过，则跳过
                if knowledge_point['id'] in processed_ids:
                    continue

                # 添加到已处理集合中，避免重复
                processed_ids.add(knowledge_point['id'])

                # 创建一个新的字典来存储当前知识点信息，包括父节点ID
                knowledge_node = {
                    'id': knowledge_point['id'],
                    'name': knowledge_point['name'],
                    'isLeaf': knowledge_point['isLeaf'],
                    'parent_id': parent_id,  # 保存父节点ID
                    'children': []  # 用于保存子节点
                }

                # 打印当前知识点信息用于调试
                print(f"知识点: {knowledge_point['name']} (ID: {knowledge_point['id']}, Parent ID: {parent_id})")

                # 如果不是叶子节点，递归获取子节点
                if not knowledge_point['isLeaf']:
                    # 递归抓取该知识点的子节点
                    knowledge_node['children'] = fetch_knowledge_points(knowledge_point['id'], cookies_file_path, processed_ids)

                # 将当前知识点加入到知识树中
                knowledge_tree.append(knowledge_node)

    return knowledge_tree

# 保存数据到 JSON 文件
def save_data_to_json(data, file_name='CPrograming_KnowledgePointTree_Raw.json'):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"数据已保存到 {file_name}")

# 使用示例
if __name__ == "__main__":
    root_parent_id = 63  # C程序设计的根节点ID
    knowledge_tree = fetch_knowledge_points(root_parent_id)

    if knowledge_tree:
        save_data_to_json(knowledge_tree)


知识点: C程序设计 (ID: 63, Parent ID: 63)
知识点: 基本数据类型与表达式 (ID: 64, Parent ID: 63)
知识点: 表达式 (ID: 65, Parent ID: 64)
知识点: 输入输出格式化控制 (ID: 66, Parent ID: 64)
知识点: 分支控制 (ID: 67, Parent ID: 63)
知识点: if-else (ID: 68, Parent ID: 67)
知识点: switch (ID: 69, Parent ID: 67)
知识点: 循环控制 (ID: 70, Parent ID: 63)
知识点: for (ID: 71, Parent ID: 70)
知识点: while和do-while (ID: 72, Parent ID: 70)
知识点: 嵌套循环 (ID: 73, Parent ID: 70)
知识点: 函数与程序结构 (ID: 74, Parent ID: 63)
知识点: 函数定义与调用 (ID: 75, Parent ID: 74)
知识点: 递归函数 (ID: 76, Parent ID: 74)
知识点: 变量作用域与存储类型 (ID: 77, Parent ID: 74)
知识点: 数组 (ID: 78, Parent ID: 63)
知识点: 一维数组 (ID: 79, Parent ID: 78)
知识点: 字符串 (ID: 80, Parent ID: 78)
知识点: 二维数组 (ID: 81, Parent ID: 78)
知识点: 指针与结构 (ID: 82, Parent ID: 63)
知识点: 指针 (ID: 83, Parent ID: 82)
知识点: 结构 (ID: 84, Parent ID: 82)
知识点: 链表 (ID: 85, Parent ID: 82)
知识点: 文件 (ID: 86, Parent ID: 63)
知识点: 文本文件处理 (ID: 87, Parent ID: 86)
知识点: 二进制文件处理 (ID: 88, Parent ID: 86)
知识点: 其他 (ID: 89, Parent ID: 63)
知识点: 编译预处理 (ID: 90, Parent ID: 89)
知识点: 命令行参数 (ID: 9

### 实现了层级关系的爬取，接下来做一些额外工作，包括爬取所有知识点的树状图

In [29]:
import requests
import pickle
import json

# 从文件中加载 Cookies
def load_cookies_from_file(cookies_file_path='cookies.pkl'):
    with open(cookies_file_path, 'rb') as f:
        cookies = pickle.load(f)
    return cookies

# 使用保存的 Cookies 发送 API 请求
def request_with_cookies(api_url, cookies_file_path='cookies.pkl'):
    # 加载 Cookies
    cookies = load_cookies_from_file(cookies_file_path)

    # 创建 session，使用保存的 cookies
    session = requests.Session()

    # 将 cookies 添加到 session
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    # 设置请求头
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json;charset=UTF-8',
        'Referer': 'https://pintia.cn/problems?knowledgePointId=63',
        'Content-Type': 'application/json;charset=UTF-8'
    }

    # 发送请求
    response = session.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"请求失败，状态码: {response.status_code}")
        return None

# 递归抓取知识点及其子知识点，并构建层级结构
def fetch_knowledge_points(parent_id, cookies_file_path='cookies.pkl', processed_ids=None):
    if processed_ids is None:
        processed_ids = set()  # 用于记录已处理的知识点ID，防止重复抓取

    api_url = f"https://pintia.cn/api/knowledge-points?parent_id={parent_id}&limit=200"
    data = request_with_cookies(api_url, cookies_file_path)
    knowledge_tree = []

    if data:
        # 遍历每个 knowledgePointPath 中的知识点
        for kp_path in data.get('knowledgePointPaths', []):
            for knowledge_point in kp_path.get('knowledgePoints', []):
                # 如果该知识点已经被处理过，则跳过
                if knowledge_point['id'] in processed_ids:
                    continue

                # 添加到已处理集合中，避免重复
                processed_ids.add(knowledge_point['id'])

                # 创建一个新的字典来存储当前知识点信息，包括父节点ID
                knowledge_node = {
                    'id': knowledge_point['id'],
                    'name': knowledge_point['name'],
                    'isLeaf': knowledge_point['isLeaf'],
                    'parent_id': parent_id,  # 保存父节点ID
                    'children': []  # 用于保存子节点
                }

                # 打印当前知识点信息用于调试
                print(f"知识点: {knowledge_point['name']} (ID: {knowledge_point['id']}, Parent ID: {parent_id})")

                # 如果不是叶子节点，递归获取子节点
                if not knowledge_point['isLeaf']:
                    # 递归抓取该知识点的子节点
                    knowledge_node['children'] = fetch_knowledge_points(knowledge_point['id'], cookies_file_path, processed_ids)

                # 将当前知识点加入到知识树中
                knowledge_tree.append(knowledge_node)

    return knowledge_tree

# 保存数据到 JSON 文件
def save_data_to_json(data, file_name='ALL_KnowledgePointTree.json.json'):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"数据已保存到 {file_name}")

# 使用示例
if __name__ == "__main__":
    root_parent_id = 0  # 从ID 0 开始爬取，表示所有的根知识点
    full_knowledge_tree = fetch_knowledge_points(root_parent_id)

    if full_knowledge_tree:
        save_data_to_json(full_knowledge_tree)


知识点: 数据结构与算法 (ID: 1, Parent ID: 0)
知识点: 算法复杂度 (ID: 2, Parent ID: 1)
知识点: 概论 (ID: 3, Parent ID: 2)
知识点: 函数的增长趋势 (ID: 4, Parent ID: 2)
知识点: 时间复杂度 (ID: 5, Parent ID: 2)
知识点: 空间复杂度 (ID: 6, Parent ID: 2)
知识点: 线性结构 (ID: 7, Parent ID: 1)
知识点: 顺序表 (ID: 8, Parent ID: 7)
知识点: 链表 (ID: 9, Parent ID: 7)
知识点: 堆栈 (ID: 10, Parent ID: 7)
知识点: 队列 (ID: 11, Parent ID: 7)
知识点: 综合 (ID: 339, Parent ID: 7)
知识点: 模式匹配 (ID: 1055603635491401728, Parent ID: 7)
知识点: 树 (ID: 12, Parent ID: 1)
知识点: 基本概念 (ID: 13, Parent ID: 12)
知识点: 二叉树及其遍历 (ID: 14, Parent ID: 12)
知识点: 二叉搜索树 (ID: 15, Parent ID: 12)
知识点: 平衡二叉树 (ID: 16, Parent ID: 12)
知识点: 完全二叉树 (ID: 17, Parent ID: 12)
知识点: 堆 (ID: 18, Parent ID: 12)
知识点: B-树和B+树 (ID: 19, Parent ID: 12)
知识点: 线索树 (ID: 20, Parent ID: 12)
知识点: 哈夫曼树 (ID: 21, Parent ID: 12)
知识点: 森林 (ID: 22, Parent ID: 12)
知识点: 并查集 (ID: 23, Parent ID: 12)
知识点: 伸展树 (ID: 24, Parent ID: 12)
知识点: 左式堆与斜堆 (ID: 25, Parent ID: 12)
知识点: 二项式队列 (ID: 26, Parent ID: 12)
知识点: 红黑树 (ID: 378, Parent ID: 12)
知识点: 线段树 (ID: 102503